In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
import pickle
import matplotlib.pyplot as plt

In [2]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    

In [3]:
def r2_prediction(regressor,X_test,y_test):
    y_pred = regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(y_test,y_pred)
    return r2

In [4]:
def Linear(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2   
    
def svm_linear(X_train,y_train,X_test):
        from sklearn.svm import SVR
        regressor = SVR(kernel='linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
    
def svm_NL(X_train,y_train,X_test):
        from sklearn.svm import SVR
        regressor = SVR(kernel='rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     
def Decision(X_train,y_train,X_test):
        # Fitting K-NN to the Training setC
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     
def random(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 
    


In [5]:
def rfeFeature(indep_X, dep_Y):
    rfelist = []

    lin = LinearRegression()
    SVRl = SVR(kernel='linear')  # Use a linear kernel for SVR
    dec = DecisionTreeRegressor(random_state=0)
    rf = RandomForestRegressor(random_state=0)

    rfemodellist = [lin,SVRl,dec,rf]
    
    for i in rfemodellist:
        log_rfe = RFE(i, n_features_to_select=3)
        log_fit = log_rfe.fit(indep_X, dep_Y)
        log_rfe_feature= log_fit.transform(indep_X)
        rfelist.append(log_rfe_feature)
    return rfelist


In [6]:
def rfe_regression(acclin,accsvml,accdes,accrf,accsvmnl): 
    rfedataframe=pd.DataFrame(index=['Linear','Random','Decision','SVR'],columns=['Linear','Random','Decision','SVRl','SVRNL'])
    
    for number,idex in enumerate(rfedataframe.index): 
            rfedataframe['Linear'][idex]=acclin[number] 
            rfedataframe['Random'][idex]=accrf[number]
            rfedataframe['Decision'][idex]=accdes[number]
            rfedataframe['SVRl'][idex]=accsvml[number]
            rfedataframe['SVRNL'][idex]=accsvmnl[number]
    return rfedataframe

In [7]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']


rfelist=rfeFeature(indep_X,dep_Y)       

acclin=[]
accsvml=[]
accsvmnl=[]
accdes=[]
accrf=[]


for i in rfelist:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y) 
    
    r2_lin=Linear(X_train,y_train,X_test)
    acclin.append(r2_lin)

    r2_sl=svm_linear(X_train,y_train,X_test)    
    accsvml.append(r2_sl)
   
    r2_snl=svm_NL(X_train,y_train,X_test)    
    accsvmnl.append(r2_snl)
    
    r2_d=Decision(X_train,y_train,X_test)
    accdes.append(r2_d)

    r2_r=random(X_train,y_train,X_test)
    accrf.append(r2_r)


    





In [8]:
result=rfe_regression(acclin,accsvml,accdes,accrf,accsvmnl)

In [9]:
result

,Linear,Random,Decision,SVRl,SVRNL
Linear,0.441961,0.441816,0.441961,0.262153,0.262162
Random,0.441961,0.441816,0.441961,0.262153,0.262162
Decision,0.664893,0.916304,0.965961,0.609652,0.883134
SVR,0.607818,0.80472,0.686535,0.58255,0.843112
